In [2]:
import requests
import json

data = {
    # "user_query": "What are total sales in 'Colefort Region'?",
    # "user_query": "List all unique regions? How many?",
    # "user_query": "What are total no. of customers per regions in our sales data?",
    # "user_query": "What are total sales by region which start with the word 'Lake'?",
    # "user_query": "How many customers are there per region?",
    # "user_query": "Count no. of unique customers in our database, and give me top 10 customers by sales in year 2025",

    # "user_query": """
    # Which product categories show consistent growth across all regions over the last 3 quarters,
    # and which regions have the highest variance in category performance?
    # """,
    # "user_query": """
    # What is the average customer lifetime value (total purchases) across different regions, and how does the distribution of high-value customers (top 10%) vary by region?
    # """,
    # "user_query": """
    # How do seasonal purchasing patterns differ between regions, and which product categories demonstrate the strongest seasonality effects?
    # """,
    # "user_query": """
    # What percentage of customers have shifted their purchasing behavior from one product category to another over the past year, and which categories show the highest customer retention?
    # Customer Retention is defined as no. of customers who continually bought the same product in last 12 months.
    # """,
    "user_query": """
    Which product pairs are frequently purchased together in each region, and how can this information identify cross-selling opportunities in underperforming regions?
    - By "frequently purchased together" I mean products that are bought together frequently in the same transaction/sale, let's say product pairs that are bought together in at least 3 sale_ids
    - I want to consider the median sales for finding the underperforming regions. A region is under performing if it is below average of median sales of all regions.
    - Do this analysis for FY 2024 and FY 2025 seperately. Here "FY" stands for full-year, not fiscal year.
    """
}

url = "http://127.0.0.1:8000/query"
lines = []
with requests.post(url, json=data, stream=True) as r:
    r.raise_for_status()
    mode = None
    for line in r.iter_lines():
        lines.append(line)
        line_dict = json.loads(line)
        if len(lines) == 1:
            thread_id = line_dict["thread_id"]
            print(f"{thread_id=}")
        model_repr = line_dict["model_repr"]
        tag = line_dict["tag"]
        response_type = line_dict["response_type"]
        response = line_dict["response"]
        if mode != (model_repr, tag, response_type):
            mode = (model_repr, tag, response_type)
            print()
            print("="*140)
            print(f"{mode=}")
            print("="*140)
        print(response, end="")

thread_id='c872f91f-148a-4553-af52-a8087dee5fa7'

mode=("ChatOllama(model='gpt-oss:20b', reasoning=True, num_ctx=20000, temperature=0.0)", 'sql_plan_generation_phase', 'reasoning')
We need to produce planner notes pseudocode/planner notes before writing SQL. No actual SQL. Must not recommend any SQL functions. Just notes. Must consider ambiguous parts? The question: "Which product pairs are frequently purchased together in each region, and how can this information identify cross-selling opportunities in underperforming regions?" They define "frequently purchased together" as product pairs that are bought together in at least 3 sale_ids. They want median sales for finding underperforming regions: region is underperforming if below average of median sales of all regions. Do analysis for FY 2024 and FY 2025 separately. FY stands for full-year, not fiscal year. So we need to consider date ranges: FY 2024: 01-Jan-2024 to 31-Dec-2024. FY 2025: 01-Jan-2025 to 31-Dec-2025. But we only have dat

In [5]:
import requests

geojsons = [
    "https://geodata.ucdavis.edu/gadm/gadm4.1/json/gadm41_PHL_0.json",
    "https://geodata.ucdavis.edu/gadm/gadm4.1/json/gadm41_PHL_1.json",
    "https://geodata.ucdavis.edu/gadm/gadm4.1/json/gadm41_PHL_2.json",
    "https://geodata.ucdavis.edu/gadm/gadm4.1/json/gadm41_PHL_3.json",
]

data = [requests.get(geojson, verify=False).json() for geojson in geojsons]

c:\Users\ujjawal.sinha\Probe\sales_bi_genai\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'geodata.ucdavis.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\ujjawal.sinha\Probe\sales_bi_genai\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'geodata.ucdavis.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\ujjawal.sinha\Probe\sales_bi_genai\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'geodata.ucdavis.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-

In [9]:
data[0].keys()

dict_keys(['type', 'name', 'crs', 'features'])

In [22]:
import numpy as np
[np.array(data[i]["features"]).shape for i in range(4)]

[(1,), (81,), (1647,), (41948,)]

In [32]:
data[0]["features"][0]["properties"]

{'GID_0': 'PHL', 'COUNTRY': 'Philippines'}